In [4]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

ar = pd.read_csv("paris_airbnb.csv")
ar.iloc[0]

stripped_commas = ar["price"].str.replace(",","")
stripped_dollar = stripped_commas.str.replace("$","")
ar["price"] = stripped_dollar.astype(float)

#ar.info()
#print(ar.head())

np.random.seed(1)
#ar = ar.loc[np.random.permutation(len(ar))]

shuffled_index = np.random.permutation(ar.index)
ar = ar.reindex(shuffled_index)

split_o = ar.iloc[0:4000]
split_t = ar.iloc[4000:]

print(ar.head())

     host_response_rate  host_acceptance_rate  host_listings_count  latitude  \
4740                NaN                   NaN                  1.0  48.88285   
5606                NaN                   NaN                  1.0  48.86006   
4824               100%                   NaN                  3.0  48.88794   
4205                60%                   NaN                  1.0  48.88868   
3228                90%                   NaN                  1.0  48.88686   

      longitude   city zipcode          state  accommodates        room_type  \
4740    2.33852  Paris   75018  Île-de-France             4  Entire home/apt   
5606    2.34768  Paris   75001  Île-de-France             3  Entire home/apt   
4824    2.34532  Paris   75018  Île-de-France             2  Entire home/apt   
4205    2.34111  Paris   75018  Île-de-France             2     Private room   
3228    2.33670  Paris   75018  Île-de-France             3  Entire home/apt   

      bedrooms  bathrooms  beds  price

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

#affectation des données one et two pour réaliser le concept de holdout, validation croisé
train_o = split_o
test_o = split_t

train_t = split_t
test_t = split_o

#première moitie pour adapter les données d'entrainement avec fit
model = KNeighborsRegressor()
model.fit(train_o[["accommodates"]],train_o["price"])
test_o["predicted_price"] = model.predict(test_o[["accommodates"]])
iteration_o_rmse = mean_squared_error(test_o["price"],test_o["predicted_price"])**(1/2)

#deuxième moitie pour adapter les données d'entrainement avec fit
model.fit(train_t[["accommodates"]],train_t["price"])
test_t["predicted_price"] = model.predict(test_t[["accommodates"]])
iteration_t_rmse = mean_squared_error(test_t["price"],test_t["predicted_price"])**(1/2)

#Moyenne des erreurs
avg_rmse = np.mean([iteration_t_rmse, iteration_o_rmse])

print(iteration_o_rmse, iteration_t_rmse, avg_rmse)


88.96592437557203 115.17976784140521 102.07284610848862


In [11]:
#Technique des K-fold, avec 4 jeux en train et 1 jeu en test et ainsi de suite avec un décalage de 1
ar["fold"] = 0
ar["fold"].iloc[0:1600] = 1
ar["fold"].iloc[1600:3200] = 2
ar["fold"].iloc[3200:4800] = 3
ar["fold"].iloc[4800:6400] = 4
ar["fold"].iloc[6400:8000] = 5

print(ar["fold"].value_counts())

3    1600
2    1600
5    1600
1    1600
4    1600
Name: fold, dtype: int64


In [14]:
model = KNeighborsRegressor()
train_iteration_o = ar[ar["fold"] != 1]
test_iteration_o = ar[ar["fold"] == 1]
model.fit(train_iteration_o[["accommodates"]],train_iteration_o["price"])

labels = model.predict(test_iteration_o[["accommodates"]])
test_iteration_o["predicted_price"] = labels

iteration_o_mse = mean_squared_error(test_iteration_o["predicted_price"],test_iteration_o["price"])
iteration_o_rmse = iteration_o_mse**(1/2)

print(iteration_o_rmse)

81.94523308283405
